In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import re
import numpy
from joblib import Parallel, delayed
tqdm.pandas()

In [2]:
%pip install tqdm_joblib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from tqdm_joblib import tqdm_joblib

c:\Users\neera\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect('eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [5]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [6]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [8]:
df.drop_duplicates(inplace = True)

In [9]:
df.isna().sum()

num        0
name       0
content    0
dtype: int64

In [10]:
b_data = df.iloc[0, 2]

print(b_data[:100])

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11'


In [11]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[385, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1')[:100])  # Assuming the content is latin-1 encoded text

1
00:00:06,000 --> 00:00:12,074
Watch any video online with Open-SUBTITLES
Free Browser extension


In [12]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [13]:
df['file_content'] = df['content'].progress_apply(decode_method)

df.head()

100%|██████████| 82498/82498 [00:43<00:00, 1894.50it/s]


,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   content       82498 non-null  object
 3   file_content  82498 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [15]:
print(df.iloc[0, 3][:100])

1
00:00:06,000 --> 00:00:12,074
Watch any video online with Open-SUBTITLES
Free Browser extension


In [16]:

# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from nltk.stem import WordNetLemmatizer


In [17]:
new_df = df[['num', 'name', 'file_content']]
new_df.columns = ['subtitle_id', 'subtitle_name', 'subtitle_content']
new_df.head()

,subtitle_id,subtitle_name,subtitle_content
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [18]:
# new_df = new_df.sample(frac = 0.2, random_state = 42)

In [19]:
new_df.shape

(82498, 3)

In [20]:
def clean_text(text):
    text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\r\n', '', text)
    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'watch any video online with opensubtitles free browser extension osdblinkext', '', text)
    text = text.strip()
    return text
# new_df['subtitle_content'] = new_df['subtitle_content'].apply(clean_text)

In [21]:
num_cores = -1 
with tqdm_joblib(total=len(new_df)) as pbar:
    cleaned_contents = Parallel(n_jobs=num_cores)(
        delayed(clean_text)(content) for content in new_df['subtitle_content']
    )
    pbar.update()

new_df['subtitle_content'] = cleaned_contents

82499it [04:39, 295.30it/s]                           
C:\Users\neera\AppData\Local\Temp\ipykernel_9924\849258213.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['subtitle_content'] = cleaned_contents


In [22]:
# new_df.reset_index(drop = True, inplace = True)
new_df['subtitle_content'][0][:100]

'in the name of god the most gracious the most merciful from muhammad the messenger of god to heracli'

In [23]:
new_df.head()

,subtitle_id,subtitle_name,subtitle_content
0,9180533,the.message.(1976).eng.1cd,in the name of god the most gracious the most ...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,yumis cells episode laptop first place is mine...
4,9180600,broker.(2022).eng.1cd,if youre going to throw it away then dont give...


In [24]:
new_df['subtitle_name'] = new_df['subtitle_name'].apply(lambda x: re.sub('\.', ' ', x))

C:\Users\neera\AppData\Local\Temp\ipykernel_9924\4139187184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['subtitle_name'] = new_df['subtitle_name'].apply(lambda x: re.sub('\.', ' ', x))


In [25]:
new_df.head()

,subtitle_id,subtitle_name,subtitle_content
0,9180533,the message (1976) eng 1cd,in the name of god the most gracious the most ...
1,9180583,here comes the grump s01 e09 joltin jack in bo...,ah theres princess dawn and terry with the blo...
2,9180592,yumis cells s02 e13 episode 2 13 (2022) eng 1cd,yumis cells episode extremely polite yumi yumi...
3,9180594,yumis cells s02 e14 episode 2 14 (2022) eng 1cd,yumis cells episode laptop first place is mine...
4,9180600,broker (2022) eng 1cd,if youre going to throw it away then dont give...


In [26]:
new_df.shape

(82498, 3)

In [27]:
def srt_files_to_chuncks(row, window_size=512, overlapping=100):
    chunks = []
    i = 0
    while i < len(row['subtitle_content']):
        chunk_end = min(i + window_size, len(row['subtitle_content']))
        chunks.append(row['subtitle_content'][i:chunk_end])
        i += window_size - overlapping
    return chunks

def process_row(row):
    return srt_files_to_chuncks(row)


In [28]:

num_cores = -1
with tqdm_joblib(total=len(new_df)) as pbar:
    processed_rows = Parallel(n_jobs=num_cores)(
        delayed(process_row)(row) for index, row in new_df.iterrows()
    )
    pbar.update()

new_df['subtitle_content'] = processed_rows

82499it [01:22, 998.13it/s]                            
C:\Users\neera\AppData\Local\Temp\ipykernel_9924\2548171154.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['subtitle_content'] = processed_rows


In [29]:
print(new_df['subtitle_content'][0][:100])

['in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with', ' about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with the call of god accept islam for your salvation embrace islam you come out of the desert smelling of camel and goat to tell persia where he should kneel muhammad messenger of god who gave him this authority god sent muhammad as a mercy to mankind the scholars and historians of islam the university of alazhar in cairo the high islamic congress of the shiat in lebanon the makers o

In [30]:
new_df['subtitle_name'] = new_df['subtitle_name'].progress_apply(lambda x: re.sub('\.', ' ', x))

100%|██████████| 82498/82498 [00:00<00:00, 180023.67it/s]
C:\Users\neera\AppData\Local\Temp\ipykernel_9924\2523961885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['subtitle_name'] = new_df['subtitle_name'].progress_apply(lambda x: re.sub('\.', ' ', x))


In [31]:
new_df.head()

,subtitle_id,subtitle_name,subtitle_content
0,9180533,the message (1976) eng 1cd,[in the name of god the most gracious the most...
1,9180583,here comes the grump s01 e09 joltin jack in bo...,[ah theres princess dawn and terry with the bl...
2,9180592,yumis cells s02 e13 episode 2 13 (2022) eng 1cd,[yumis cells episode extremely polite yumi yum...
3,9180594,yumis cells s02 e14 episode 2 14 (2022) eng 1cd,[yumis cells episode laptop first place is min...
4,9180600,broker (2022) eng 1cd,[if youre going to throw it away then dont giv...


In [32]:
with tqdm(total=1) as pbar:
    new_df = new_df.explode('subtitle_content', ignore_index=False)
    pbar.update()

100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [33]:
new_df.shape

(5197326, 3)

In [34]:
#new_df.to_csv('main_search_engine.csv', index = False)

In [35]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer,util

In [50]:
client = chromadb.PersistentClient(path="C:\\Users\\neera\\OneDrive\\Desktop\\Innomatics_Labs_data_science\\Semantic_Search_Engine_Final_Project\\db_files")

In [51]:
client.heartbeat()

1714019557344042600

In [52]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [39]:
collection = client.create_collection(
        name="subtitles_embedded",
        metadata={"hnsw:space": "cosine"},
        embedding_function=sentence_transformer_ef
    )

In [40]:
# df_for_em = new_df.sample(frac = 0.08, random_state = 42, axis = 0)
df_for_em = new_df

In [41]:
df_for_em.shape

(5197326, 3)

In [42]:
# df_for_em.reset_index(drop = True, inplace = True)

In [43]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [44]:
df_with_em = df_for_em 

In [45]:
def encoding_content(x):
    return model.encode(x, normalize_embeddings=True).tolist()

# df_with_em.loc[:, 'subtitle_content_encoded'] = df_for_em['subtitle_content'].progress_apply(encoding_content)

In [46]:
from joblib import parallel_backend

In [47]:
# num_cores = -1  
# with tqdm_joblib(total=len(df_for_em)) as pbar:
#     encoded_contents = Parallel(n_jobs=num_cores)(
#         delayed(encoding_content)(content) for content in df_for_em['subtitle_content']
#     )
#     pbar.update()
# df_with_em['subtitle_content_encoded'] = encoded_contents

In [49]:
num_cores = -1
with parallel_backend('threading', n_jobs=num_cores):
    encoded_contents = Parallel()(delayed(encoding_content)(content) for content in tqdm(df_for_em['subtitle_content']))

df_with_em['subtitle_content_encoded'] = encoded_contents

In [ ]:
df_with_em.info()

In [ ]:
df_with_em.head()

In [ ]:
for i in tqdm(range(df_with_em.shape[0])):
    collection.add(
        documents=df_with_em['subtitle_content'].iloc[i],
        embeddings=df_with_em['subtitle_content_encoded'].iloc[i],
        metadatas= {'subtitle_name': df_with_em['subtitle_name'].iloc[i], 'subtitle_id': str(df_with_em['subtitle_id'].iloc[i])},
        ids=str(i)
    )

In [ ]:
import random
temp = df_with_em.iloc[random.randint(0, len(df_with_em))]
for i in range(4):
    if i == 3:
        continue
    else:
        print(temp[i])

In [62]:
collection = client.get_collection(name="subtitles_embedded", embedding_function=sentence_transformer_ef)

In [63]:
# query = "ross you know yesterday he asked me to fix him up with somebody"
query = "hey ross how are you"

query = clean_text(query)

In [65]:
query_em = encoding_content(query)

In [66]:
search_results = collection.query(query_embeddings=query_em, n_results=5)

In [67]:
print("Top 5 Results")
for i, res in enumerate(search_results['metadatas'][0]):
    print(f"Result {i+1} : ", res['subtitle_name']) 


Top 5 Results
Result 1 :  friends s02 e01 the one with rosss new girlfriend (1995) eng 1cd
Result 2 :  friends s08 e04 the one with the videotape (2001) eng 1cd
Result 3 :  friends s10 e17 the last one (2004) eng 1cd
Result 4 :  friends s06 e13 the one with rachels sister (2000) eng 1cd
Result 5 :  friends s09 e14 the one with the blind dates (2003) eng 1cd
